In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm import tqdm_notebook

In [2]:
# Load credentials for Yelp API
with open('/Users/hamma/.secret/yelp_api.json') as f:
    login = json.load(f)
    
# Instantiate Yelp API variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# Set variables for API params
location = 'NY, NY'
term = 'Pizza'

In [4]:
# Create results in progress json file
json_file = 'results_in_progress_NY_pizza.json'
json_file

'results_in_progress_NY_pizza.json'

In [5]:
# Check if file exists
file_exist = os.path.isfile(json_file)

#If does not exist
if file_exist == False:
    
    # Create needed folder
    folder = os.path.dirname(json_file)
    
    # If json_file included a folder
    if len(folder) > 0:
        os.makedirs(folder, exist_ok=True)
    
    # Inform user and save empty list
    print(f'[i] {json_file} not found. Saving empty list to file')
    
    # Save empty list
    with open(json_file, 'w') as f:
        json.dump([], f)
    
# If it exists inform the user
else:
    print(f'[i] {json_file} already exists')

[i] results_in_progress_NY_pizza.json already exists


In [6]:
# Load previous results and use len of results for offset
with open(json_file, 'r') as f:
    previous_results = json.load(f)

# Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found')

- 0 previous results found


In [7]:
# Use search query method to perform API call
results = yelp_api.search_query(location=location, term=term, offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
total_results = results['total']
total_results

16000

In [9]:
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Use math.ceil to round up the total number fo pages of results
n_pages = math.ceil((results['total'] - n_results) / results_per_page)
n_pages

800

In [11]:
# Add first page of results to empty list
previous_results.extend(results['businesses'])
with open(json_file, 'w') as f:
    json.dump(previous_results, f)

In [12]:
# Add progress bar
for i in tqdm_notebook(range(n_pages)):
    time.sleep(0.2)

C:\Users\hamma\AppData\Local\Temp\ipykernel_3400\1813211223.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n_pages)):


  0%|          | 0/800 [00:00<?, ?it/s]

In [13]:
# For loop to call each page
for i in tqdm_notebook(range(1,n_pages+1)):
    
    # Read in results and check length
    with open(json_file, 'r') as f:
        previous_results = json.load(f)
    
    # Save number of results to use as offset
    n_results = len(previous_results)
    
    # Use n_results as offset
    results = yelp_api.search_query(location=location, term=term,
                                   offset=n_results)
    
    # Append new results and save file
    previous_results.extend(results['businesses'])
    
    with open(json_file, 'w') as f:
        json.dump(previous_results, f)
        
    # add 200ms pause
    time.sleep(0.2)

C:\Users\hamma\AppData\Local\Temp\ipykernel_3400\4143907267.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1,n_pages+1)):


  0%|          | 0/800 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [14]:
# Delete old file ad confirm it was deleted
os.remove(json_file)
os.path.isfile(json_file)

False

In [22]:
# Create function to create json files
def create_json_file(json_file, delete_if_exists=False):
    
    # Check if file exists
    file_exists = os.path.isfile(json_file)
    
    # If it does exist
    if file_exists == True:
        
        # Check if user wants to delete if exists
        if delete_if_exists==True:
            print(f'[!] {json_file} already exists. Deleting previous file')
        
            # Delete file and confirm it no longer exists
            os.remove(json_file)
            print(f'[i] {json_file} already exists')
            
            # Recursive call to function after old file delete
            create_json_file(json_file, delete_if_exists=False)
        else:
            print(f'[i] {json_file} already exists')
    
    # If it does not exist
    else:
        # Inform user and save empty list
        print(f'[i] {json_file} not found. Saving empty list to new file')
    
        # Create any needed folders
        folder = os.path.dirname(json_file)
    
        # If included a folder
        if len(folder) > 0:
          
            # Create folder
            os.makdirs(folder, exist_ok=True)
    
        # Save empty list to start the json file
        with open(json_file, 'w') as f:
            json.dump([], f)

In [23]:
# Create new empty json file
create_json_file(json_file, delete_if_exists=True)

# Load previous rsults and use len of results for offset
with open(json_file, 'r') as f:
    previous_results = json.load(f)

# Set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found')

# Use search query method to perform API call
results = yelp_api.search_query(location=location, term=term, offset=n_results)

# How many results in total?
total_results = results['total']

# How many did we get details for?
results_per_page = len(results['businesses'])

# Use math.ceil to round up for the total number of pages of results
n_pages = math.ceil((total_results - n_results) / results_per_page)
n_pages

[!] results_in_progress_NY_pizza.json already exists. Deleting previous file
[i] results_in_progress_NY_pizza.json already exists
[i] results_in_progress_NY_pizza.json not found. Saving empty list to new file
- 0 previous results found


740

In [24]:
for i in tqdm_notebook(range(1, n_pages+1)):
    
    # Read in results in progress file and check length
    with open(json_file, 'r') as f:
        previous_results = json.load(f)
        
    # Save number of results to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls, stopping loop')
        break
    
    # Use n_results as the offset
    results = yelp_api.search_query(location=location, term=term, 
                                    offset=n_results)
    
    # Append new results and save to file
    previous_results.extend(results['businesses'])
    
    # Display(previous_results)
    with open(json_file, 'w') as f:
        json.dump(previous_results, f)
        
    time.sleep(0.2)

C:\Users\hamma\AppData\Local\Temp\ipykernel_3400\1547159822.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(1, n_pages+1)):


  0%|          | 0/740 [00:00<?, ?it/s]

Exceeded 1000 api calls, stopping loop


In [25]:
# Load final results
final_df = pd.read_json(json_file)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2634,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4859,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/F65qqO...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2984,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/KKB2Tr...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,934,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/m_9TT6...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6561,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,2133.973859


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,ZZaBt_paohbRRm_Gjb0bhw,dugout-breezy-point,Dugout,,False,https://www.yelp.com/biz/dugout-breezy-point?a...,5,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.5593119845147, 'longitude': -7...",[],NaN,"{'address1': '57 Point Breeze Ave', 'address2'...",+17186346776,(718) 634-6776,17168.017817
996,XbUz43jqdNQ7esCWPS-j_A,avenue-pizza-bloomfield,Avenue Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/4-5HsS...,False,https://www.yelp.com/biz/avenue-pizza-bloomfie...,33,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7762219, 'longitude': -74.1874...","[pickup, delivery]",$$,"{'address1': '32 Bloomfield Ave', 'address2': ...",+19734041486,(973) 404-1486,19170.832237
997,KMA2hqRAcjHa3QMJ5ynz0g,ipizzany-new-york-8,iPizzaNY,https://s3-media1.fl.yelpcdn.com/bphoto/YdmrQH...,False,https://www.yelp.com/biz/ipizzany-new-york-8?a...,10,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.76077, 'longitude': -73.99071}",[],NaN,"{'address1': '656 9th Ave', 'address2': '', 'a...",,,6057.477800
998,TWxs38VmW0O7K3LgT1K8qg,bklyn-pizza-bedford-stuyvesant-4,Bklyn Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UtKmRp...,False,https://www.yelp.com/biz/bklyn-pizza-bedford-s...,173,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.68903, 'longitude': -73.92408}","[pickup, delivery]",$,"{'address1': '36 Ralph Ave', 'address2': '', '...",+17184848787,(718) 484-8787,5053.087959
999,PJff4lIQtTaEJhY2mWsCnA,casabianca-pizzeria-new-york,Casabianca Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/dBn3-0...,False,https://www.yelp.com/biz/casabianca-pizzeria-n...,60,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 40.7684632, 'longitude': -73.9886...","[pickup, delivery]",$,"{'address1': '854 Tenth Ave', 'address2': '', ...",+12123978725,(212) 397-8725,6883.423370


In [26]:
# Check for duplicates
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [28]:
# Check for duplicate IDs
final_df.duplicated(subset='id').sum()

236

In [29]:
# Drop duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [30]:
# Save final results to a compressed csv
final_df.to_csv('final_results_NY_pizza.csv.gz', compression='gzip', 
                index=False)